# Searching for DKIST Datasets

In this session we will cover how to search for DKIST datasets available at the DKIST Data Center.
In DKIST data parlance, a "dataset" is the smallest unit of data that is searchable from the data centre, and represents a single observation from a single instrument at a single pass band.

Each dataset comprises a number of different files:
  * An ASDF file containing all the metadata, and no data.
  * A quality report PDF.
  * An mp4 preview movie.
  * A (large) number of FITS files, each containing a "calibrated exposure".

All of these files apart from the FITS files containing the data can be downloaded irrespective of embargo status.

For each of these "datasets" the DKIST Data Center keeps a "dataset inventory record" which is a limited set of metadata about the dataset on which you can search, either through the portal or the `dkist` Python package.
The ASDF, quality report and preview movie can all be downloaded without authenticating, the FITS files require the use of Globus.


## Using `Fido.search`

The search interface we are going to use is {obj}`sunpy.net.Fido`.
`Fido` supports many different sources of data, some built into `sunpy` like the VSO and some in plugins like `dkist` or `sunpy-soar`.
With `Fido` you can search for DKIST datasets and download their corresponding ASDF files.
To register the DKIST search with `Fido` we must also import `dkist.net`.

In [1]:
import astropy.units as u
from sunpy.net import Fido, attrs as a
import dkist.net

`Fido` searches are built up from "attrs", which we imported above as `a`.
These attrs are combined together with either logical AND or logical OR operations to make complex queries.
Let's start simple and search for all the DKIST datasets which are not embargoed:

In [4]:
Fido.search(a.dkist.Embargoed(False))

Start Time,End Time,Instrument,Wavelength,Bounding Box,Dataset ID,Dataset Size,Exposure Time,Primary Experiment ID,Primary Proposal ID,Stokes Parameters,Target Types,Number of Frames,Average Fried Parameter,Embargoed,Downloadable,Has Spectral Axis,Has Temporal Axis,Average Spectral Sampling,Average Spatial Sampling,Average Temporal Sampling
,,,nm,,,Gibyte,s,,,,,,,,,,,nm,arcsec,s
Time,Time,str4,float64[2],str36,str5,float64,float64,str9,str9,str4,str7[1],int64,float64,bool,bool,bool,bool,float64,float64,float64
2022-06-02T17:22:50.176,2022-06-02T17:47:30.856,VBI,486.0 .. 486.0,"(-560.88,-346.83),(-677.78,-466.44)",BLKGA,2.0,5.009,eid_1_118,pid_1_118,I,unknown,171,0.08814365069897488,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:21:55.346,2022-06-02T17:47:58.286,VBI,393.0 .. 393.0,"(-560.88,-346.83),(-677.79,-466.44)",BJLKB,2.0,25.007,eid_1_118,pid_1_118,I,unknown,180,0.09129593540153651,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:22:22.754,2022-06-02T17:48:25.694,VBI,430.0 .. 430.0,"(-560.88,-346.83),(-677.78,-466.44)",BKJYA,2.0,1.005,eid_1_118,pid_1_118,I,unknown,180,0.09618521236484806,False,True,False,True,nan,0.01099999994039536,82.25999999999694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03T17:38:57.655,2022-06-03T18:05:45.695,VISP,854.248985847081 .. 856.7855075464167,"(-379.32,97931.48),(-435.8,1727.87)",AVORO,4.0,48.00811267605634,eid_1_118,pid_1_118,IQUV,unknown,1960,0.14324949289555358,False,True,True,True,0.000999811469978602,0.2134568481952311,804.0201897959191
2022-06-02T18:50:16.655,2022-06-02T19:17:04.695,VISP,854.248985835402 .. 856.7855075347377,"(-408.32,97828.51),(-462.36,1624.88)",ALGJX,4.0,48.00811267605634,eid_1_118,pid_1_118,IQUV,unknown,1960,0.08793694602367085,False,True,True,True,0.000999811469978602,0.2134568481952311,804.0203285714256
2022-10-24T18:57:45.634,2022-10-24T19:33:26.865,VISP,630.2424776472172 .. 631.826964866207,"(233.36,-368.87),(212.1,-503.97)",BKEWK,5.0,48.00811267605634,eid_1_123,pid_1_123,IQUV,unknown,2752,0.11445240041497136,False,True,True,True,0.00162511509639976,0.04160701928854328,1070.6156795058157


Because we only specified one attr, and it was unique to the dkist client (it started with `a.dkist`) only the DKIST client was used.

If we only want VBI datasets, that are unembargoed, between a specific time range we can use multiple attrs:

In [13]:
Fido.search(a.Time("2022-06-02 17:00:00", "2022-06-02 18:00:00") & a.Instrument("VBI") & a.dkist.Embargoed(False))

Start Time,End Time,Instrument,Wavelength,Bounding Box,Dataset ID,Dataset Size,Exposure Time,Primary Experiment ID,Primary Proposal ID,Stokes Parameters,Target Types,Number of Frames,Average Fried Parameter,Embargoed,Downloadable,Has Spectral Axis,Has Temporal Axis,Average Spectral Sampling,Average Spatial Sampling,Average Temporal Sampling
,,,nm,,,Gibyte,s,,,,,,,,,,,nm,arcsec,s
Time,Time,str3,float64[2],str35,str5,float64,float64,str9,str9,str1,str7[1],int64,float64,bool,bool,bool,bool,float64,float64,float64
2022-06-02T17:22:50.176,2022-06-02T17:47:30.856,VBI,486.0 .. 486.0,"(-560.88,-346.83),(-677.78,-466.44)",BLKGA,2.0,5.009,eid_1_118,pid_1_118,I,unknown,171,0.08814365069897488,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:21:55.346,2022-06-02T17:47:58.286,VBI,393.0 .. 393.0,"(-560.88,-346.83),(-677.79,-466.44)",BJLKB,2.0,25.007,eid_1_118,pid_1_118,I,unknown,180,0.09129593540153651,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:22:22.754,2022-06-02T17:48:25.694,VBI,430.0 .. 430.0,"(-560.88,-346.83),(-677.78,-466.44)",BKJYA,2.0,1.005,eid_1_118,pid_1_118,I,unknown,180,0.09618521236484806,False,True,False,True,nan,0.01099999994039536,82.25999999999694


Note how the `a.Time` and `a.Instrument` attrs are not prefixed with `dkist` - these are general attrs which can be used to search multiple sources of data.

So far all returned results have had to match all the attrs provided, because we have used the `&` (logical AND) operator to join them.
If we want results that match either one of multiple options we can use the `|` operator.

In [16]:
time = a.Time("2022-06-02 17:00", "2022-06-02 18:00")
visp = a.Instrument("VISP") & a.dkist.Embargoed(False)
aia = a.Instrument.aia & a.Wavelength(17.1 * u.nm) & a.Sample(30 * u.min)

Fido.search(time, aia | visp)

<sunpy.net.fido_factory.UnifiedResponse object at 0x7f77bca80750>
Results from 2 Providers:

2 Results from the VSOClient:
Source: http://vso.stanford.edu/cgi-bin/search
Total estimated size: 135.578 Mbyte

       Start Time               End Time        Source Instrument   Wavelength   Provider  Physobs  Wavetype Extent Width Extent Length Extent Type   Size  
                                                                     Angstrom                                                                        Mibyte 
----------------------- ----------------------- ------ ---------- -------------- -------- --------- -------- ------------ ------------- ----------- --------
2022-06-02 17:00:09.000 2022-06-02 17:00:10.000    SDO        AIA 171.0 .. 171.0     JSOC intensity   NARROW         4096          4096    FULLDISK 64.64844
2022-06-02 17:30:09.000 2022-06-02 17:30:10.000    SDO        AIA 171.0 .. 171.0     JSOC intensity   NARROW         4096          4096    FULLDISK 64.64844

2 Results from the DKISTClient:

       Start Time               End Time        Instrument               Wavelength               ... Average Spectral Sampling Average Spatial Sampling Average Temporal Sampling
                                                                             nm                   ...             nm                     arcsec                      s            
----------------------- ----------------------- ---------- -------------------------------------- ... ------------------------- ------------------------ -------------------------
2022-06-02T17:21:59.648 2022-06-02T17:48:47.688       VISP 630.2392098893586 .. 634.3897538455635 ...       0.00162511509639976       0.2134568481952311         804.0201081632624
2022-06-02T17:21:59.648 2022-06-02T17:48:47.688       VISP  854.248985823001 .. 856.7855075223367 ...      0.000999811469978602       0.2134568481952311         804.0201081632624

As you can see this has returned two separate tables, one for VBI and one for VISP.

Because `Fido` can search other clients as well as the DKIST you can make a more complex query which will search for VISP data and context images from AIA at the same time:

Here we have used a couple of different attrs.
`a.Sample` limits the results to one per time window given, and `a.Wavelength` searches for specific wavelengths of data.
Also, we passed our attrs as positional arguments to `Fido.search`.
This is a little bit of sugar to prevent having to specify a lot of brackets; all arguments have the and (`&`) operator applied to them.

## Working with Results Tables

A Fido search returns a {obj}`sunpy.net.fido_factory.UnifiedResponse` object, which contains all the search results from all the different clients and requests made to the servers.

In [18]:
res = Fido.search((a.Instrument.vbi | a.Instrument.visp) & a.dkist.Embargoed(False))
res

Start Time,End Time,Instrument,Wavelength,Bounding Box,Dataset ID,Dataset Size,Exposure Time,Primary Experiment ID,Primary Proposal ID,Stokes Parameters,Target Types,Number of Frames,Average Fried Parameter,Embargoed,Downloadable,Has Spectral Axis,Has Temporal Axis,Average Spectral Sampling,Average Spatial Sampling,Average Temporal Sampling
,,,nm,,,Gibyte,s,,,,,,,,,,,nm,arcsec,s
Time,Time,str3,float64[2],str35,str5,float64,float64,str9,str9,str1,str7[1],int64,float64,bool,bool,bool,bool,float64,float64,float64
2022-06-02T17:22:50.176,2022-06-02T17:47:30.856,VBI,486.0 .. 486.0,"(-560.88,-346.83),(-677.78,-466.44)",BLKGA,2.0,5.009,eid_1_118,pid_1_118,I,unknown,171,0.08814365069897488,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:21:55.346,2022-06-02T17:47:58.286,VBI,393.0 .. 393.0,"(-560.88,-346.83),(-677.79,-466.44)",BJLKB,2.0,25.007,eid_1_118,pid_1_118,I,unknown,180,0.09129593540153651,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:22:22.754,2022-06-02T17:48:25.694,VBI,430.0 .. 430.0,"(-560.88,-346.83),(-677.78,-466.44)",BKJYA,2.0,1.005,eid_1_118,pid_1_118,I,unknown,180,0.09618521236484806,False,True,False,True,nan,0.01099999994039536,82.25999999999694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03T17:09:55.346,2022-06-03T17:35:58.285,VBI,393.0 .. 393.0,"(-351.85,-375.79),(-468.81,-495.48)",BDOVA,2.0,25.007,eid_1_118,pid_1_118,I,unknown,180,0.11833839105343869,False,True,False,True,nan,0.01099999994039536,82.25999899999685
2022-06-03T17:10:22.753,2022-06-03T17:36:25.693,VBI,430.0 .. 430.0,"(-351.85,-375.79),(-468.81,-495.48)",BEQYB,2.0,0.6970000000000001,eid_1_118,pid_1_118,I,unknown,180,0.12242215601100881,False,True,False,True,nan,0.01099999994039536,82.25999999999694
2022-06-03T17:10:50.175,2022-06-03T17:35:30.855,VBI,486.0 .. 486.0,"(-351.85,-375.79),(-468.82,-495.48)",AGXWB,2.0,3.997000000000001,eid_1_118,pid_1_118,I,unknown,171,0.12313722407803035,False,True,False,True,nan,0.01099999994039536,82.25999899998726
Start Time,End Time,Instrument,Wavelength,Bounding Box,Dataset ID,Dataset Size,Exposure Time,Primary Experiment ID,Primary Proposal ID,Stokes Parameters,Target Types,Number of Frames,Average Fried Parameter,Embargoed,Downloadable,Has Spectral Axis,Has Temporal Axis,Average Spectral Sampling,Average Spatial Sampling,Average Temporal Sampling


The `UnifiedResponse` object provides a couple of different ways to select the results you are interested in.
It's possible to select just the results returned by a specific client by name, in this case all the results are from the DKIST client so this changes nothing.

In [19]:
res["dkist"]

Start Time,End Time,Instrument,Wavelength,Bounding Box,Dataset ID,Dataset Size,Exposure Time,Primary Experiment ID,Primary Proposal ID,Stokes Parameters,Target Types,Number of Frames,Average Fried Parameter,Embargoed,Downloadable,Has Spectral Axis,Has Temporal Axis,Average Spectral Sampling,Average Spatial Sampling,Average Temporal Sampling
,,,nm,,,Gibyte,s,,,,,,,,,,,nm,arcsec,s
Time,Time,str3,float64[2],str35,str5,float64,float64,str9,str9,str1,str7[1],int64,float64,bool,bool,bool,bool,float64,float64,float64
2022-06-02T17:22:50.176,2022-06-02T17:47:30.856,VBI,486.0 .. 486.0,"(-560.88,-346.83),(-677.78,-466.44)",BLKGA,2.0,5.009,eid_1_118,pid_1_118,I,unknown,171,0.08814365069897488,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:21:55.346,2022-06-02T17:47:58.286,VBI,393.0 .. 393.0,"(-560.88,-346.83),(-677.79,-466.44)",BJLKB,2.0,25.007,eid_1_118,pid_1_118,I,unknown,180,0.09129593540153651,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:22:22.754,2022-06-02T17:48:25.694,VBI,430.0 .. 430.0,"(-560.88,-346.83),(-677.78,-466.44)",BKJYA,2.0,1.005,eid_1_118,pid_1_118,I,unknown,180,0.09618521236484806,False,True,False,True,nan,0.01099999994039536,82.25999999999694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03T17:09:55.346,2022-06-03T17:35:58.285,VBI,393.0 .. 393.0,"(-351.85,-375.79),(-468.81,-495.48)",BDOVA,2.0,25.007,eid_1_118,pid_1_118,I,unknown,180,0.11833839105343869,False,True,False,True,nan,0.01099999994039536,82.25999899999685
2022-06-03T17:10:22.753,2022-06-03T17:36:25.693,VBI,430.0 .. 430.0,"(-351.85,-375.79),(-468.81,-495.48)",BEQYB,2.0,0.6970000000000001,eid_1_118,pid_1_118,I,unknown,180,0.12242215601100881,False,True,False,True,nan,0.01099999994039536,82.25999999999694
2022-06-03T17:10:50.175,2022-06-03T17:35:30.855,VBI,486.0 .. 486.0,"(-351.85,-375.79),(-468.82,-495.48)",AGXWB,2.0,3.997000000000001,eid_1_118,pid_1_118,I,unknown,171,0.12313722407803035,False,True,False,True,nan,0.01099999994039536,82.25999899998726
Start Time,End Time,Instrument,Wavelength,Bounding Box,Dataset ID,Dataset Size,Exposure Time,Primary Experiment ID,Primary Proposal ID,Stokes Parameters,Target Types,Number of Frames,Average Fried Parameter,Embargoed,Downloadable,Has Spectral Axis,Has Temporal Axis,Average Spectral Sampling,Average Spatial Sampling,Average Temporal Sampling


This object is similar to a list of tables, where each response can also be selected by the first index:

In [23]:
vbi = res[0]
vbi

Start Time,End Time,Instrument,Wavelength,Bounding Box,Dataset ID,Dataset Size,Exposure Time,Primary Experiment ID,Primary Proposal ID,Stokes Parameters,Target Types,Number of Frames,Average Fried Parameter,Embargoed,Downloadable,Has Spectral Axis,Has Temporal Axis,Average Spectral Sampling,Average Spatial Sampling,Average Temporal Sampling
,,,nm,,,Gibyte,s,,,,,,,,,,,nm,arcsec,s
Time,Time,str3,float64[2],str35,str5,float64,float64,str9,str9,str1,str7[1],int64,float64,bool,bool,bool,bool,float64,float64,float64
2022-06-02T17:22:50.176,2022-06-02T17:47:30.856,VBI,486.0 .. 486.0,"(-560.88,-346.83),(-677.78,-466.44)",BLKGA,2.0,5.009,eid_1_118,pid_1_118,I,unknown,171,0.08814365069897488,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:21:55.346,2022-06-02T17:47:58.286,VBI,393.0 .. 393.0,"(-560.88,-346.83),(-677.79,-466.44)",BJLKB,2.0,25.007,eid_1_118,pid_1_118,I,unknown,180,0.09129593540153651,False,True,False,True,nan,0.01099999994039536,82.26000000000653
2022-06-02T17:22:22.754,2022-06-02T17:48:25.694,VBI,430.0 .. 430.0,"(-560.88,-346.83),(-677.78,-466.44)",BKJYA,2.0,1.005,eid_1_118,pid_1_118,I,unknown,180,0.09618521236484806,False,True,False,True,nan,0.01099999994039536,82.25999999999694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03T17:09:55.346,2022-06-03T17:35:58.285,VBI,393.0 .. 393.0,"(-351.85,-375.79),(-468.81,-495.48)",BDOVA,2.0,25.007,eid_1_118,pid_1_118,I,unknown,180,0.11833839105343869,False,True,False,True,nan,0.01099999994039536,82.25999899999685
2022-06-03T17:10:22.753,2022-06-03T17:36:25.693,VBI,430.0 .. 430.0,"(-351.85,-375.79),(-468.81,-495.48)",BEQYB,2.0,0.6970000000000001,eid_1_118,pid_1_118,I,unknown,180,0.12242215601100881,False,True,False,True,nan,0.01099999994039536,82.25999999999694
2022-06-03T17:10:50.175,2022-06-03T17:35:30.855,VBI,486.0 .. 486.0,"(-351.85,-375.79),(-468.82,-495.48)",AGXWB,2.0,3.997000000000001,eid_1_118,pid_1_118,I,unknown,171,0.12313722407803035,False,True,False,True,nan,0.01099999994039536,82.25999899998726


Now we have selected a single set of results from the `UnifiedResponse` object, we can see that we have a `DKISTQueryResponseTable` object:

In [24]:
type(vbi)

dkist.net.client.DKISTQueryResponseTable

This is a subclass of {obj}`astropy.table.QTable`, which means we can do operations such as sorting and filtering with this table.

We can display only some columns:

In [25]:
vbi["Dataset ID", "Start Time", "Average Fried Parameter", "Embargoed"]

Dataset ID,Start Time,Average Fried Parameter,Embargoed
str5,Time,float64,bool
BLKGA,2022-06-02T17:22:50.176,0.08814365069897488,False
BJLKB,2022-06-02T17:21:55.346,0.09129593540153651,False
BKJYA,2022-06-02T17:22:22.754,0.09618521236484806,False
...,...,...,...
AXDJA,2022-06-03T17:38:57.346,0.13728811110119296,False
BDOVA,2022-06-03T17:09:55.346,0.11833839105343869,False
BEQYB,2022-06-03T17:10:22.753,0.12242215601100881,False
AGXWB,2022-06-03T17:10:50.175,0.12313722407803035,False


or sort based on a column, and pick the top 5 rows:

In [30]:
vbi.sort("Average Fried Parameter", reverse=True)
vbi[:5]["Dataset ID", "Average Fried Parameter"]

Dataset ID,Average Fried Parameter
str5,float64
BXZJB,2.2476176938783308
BWZMA,2.099377954250577
AKGYA,1.7037129381032616
AMKYA,1.6382080477720182
BYEPB,1.4084623247425094


Once we have selected the rows we are interested in we can move onto downloading the ASDF files.

## Downloading Files with `Fido.fetch`

```{note}
Only the ASDF files, and not the FITS files containing the data can be downloaded with `Fido`.
```

To download files with `Fido` we pass the search results to `Fido.fetch`.

If we want to download the first VBI file we searched for earlier we can do so like this:

In [37]:
Fido.fetch(vbi[:5])

Files Downloaded:   0%|          | 0/5 [00:00<?, ?file/s]

VBI_L1_20220602T201849_AMKYA.asdf:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

VBI_L1_20220602T201754_AKGYA.asdf:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

VBI_L1_20220603T203203_BWZMA.asdf:   0%|          | 0.00/1.76M [00:00<?, ?B/s]

VBI_L1_20220603T203258_BYEPB.asdf:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

['/home/stuart/sunpy/data/VBI_L1_20220603T203230_BXZJB.asdf', '/home/stuart/sunpy/data/VBI_L1_20220603T203203_BWZMA.asdf', '/home/stuart/sunpy/data/VBI_L1_20220602T201754_AKGYA.asdf', '/home/stuart/sunpy/data/VBI_L1_20220602T201849_AMKYA.asdf', '/home/stuart/sunpy/data/VBI_L1_20220603T203258_BYEPB.asdf']

This will download the ASDF file to the sunpy default data directory `~/sunpy/data`, we can customise this with the `path=` keyword argument.

A simple example of specifying the path is:

In [39]:
Fido.fetch(vbi[0], path=".")

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

VBI_L1_20220603T203230_BXZJB.asdf:   0%|          | 0.00/1.76M [00:00<?, ?B/s]

['VBI_L1_20220603T203230_BXZJB.asdf']

This will download the ASDF file as `/data/mypath/filename.asdf`.

With the nature of DKIST data being a large number of files - FITS + ASDF for a whole dataset - we probably want to keep each dataset in it's own folder.
`Fido` makes this easy by allowing you to provide a path template rather than a specific path.
To see the list of parameters we can use in these path templates we can run:

In [40]:
vbi.path_format_keys()

{'asdf_filename',
 'average_fried_parameter',
 'average_spatial_sampling',
 'average_spectral_sampling',
 'average_temporal_sampling',
 'bounding_box',
 'calibration_documentation_url',
 'calibration_workflow_name',
 'calibration_workflow_version',
 'creation_date',
 'dataset_id',
 'dataset_size',
 'downloadable',
 'embargoed',
 'end_time',
 'experiment_description',
 'experiment_ids',
 'exposure_time',
 'full_stokes',
 'has_spectral_axis',
 'has_temporal_axis',
 'hdu_creation_date',
 'header_documentation_url',
 'header_specification_version',
 'info_url',
 'input_dataset_observe_frames_part_id',
 'input_dataset_parameters_part_id',
 'instrument',
 'instrument_program_execution_id',
 'last_updated',
 'level_0_frame_count',
 'movie_filename',
 'number_of_frames',
 'observing_program_execution_id',
 'preview_url',
 'primary_experiment_id',
 'primary_proposal_id',
 'proposal_ids',
 'quality_report_filename',
 'recipe_id',
 'recipe_instance_id',
 'recipe_run_id',
 'start_time',
 'stokes_p

So if we want to put each of our ASDF files in a directory named with the Dataset ID and Instrument we can do:

In [46]:
Fido.fetch(vbi[:5], path="~/sunpy/data/{instrument}/{dataset_id}", overwrite=True)

Files Downloaded:   0%|          | 0/5 [00:00<?, ?file/s]

VBI_L1_20220602T201754_AKGYA.asdf:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

VBI_L1_20220602T201849_AMKYA.asdf:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

VBI_L1_20220603T203203_BWZMA.asdf:   0%|          | 0.00/1.76M [00:00<?, ?B/s]

VBI_L1_20220603T203230_BXZJB.asdf:   0%|          | 0.00/1.76M [00:00<?, ?B/s]

VBI_L1_20220603T203258_BYEPB.asdf:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

['/home/stuart/sunpy/data/VBI/BXZJB/VBI_L1_20220603T203230_BXZJB.asdf', '/home/stuart/sunpy/data/VBI/BWZMA/VBI_L1_20220603T203203_BWZMA.asdf', '/home/stuart/sunpy/data/VBI/AKGYA/VBI_L1_20220602T201754_AKGYA.asdf', '/home/stuart/sunpy/data/VBI/AMKYA/VBI_L1_20220602T201849_AMKYA.asdf', '/home/stuart/sunpy/data/VBI/BYEPB/VBI_L1_20220603T203258_BYEPB.asdf']

In [50]:
vbi.colnames

['Start Time',
 'End Time',
 'Instrument',
 'Wavelength',
 'Bounding Box',
 'Movie Filename',
 'Storage Bucket',
 'Dataset ID',
 'Dataset Size',
 'Experiment IDs',
 'Exposure Time',
 'Level 0 Frame count',
 'Primary Experiment ID',
 'Primary Proposal ID',
 'Proposal IDs',
 'Recipe Instance ID',
 'Recipe Run ID',
 'Recipe ID',
 'Full Stokes',
 'Stokes Parameters',
 'Target Types',
 'Creation Date',
 'Number of Frames',
 'Average Fried Parameter',
 'asdf Filename',
 'Experiment Description',
 'Embargoed',
 'Last Updated',
 'Preview URL',
 'Downloadable',
 'Has Spectral Axis',
 'Has Temporal Axis',
 'Average Spectral Sampling',
 'Average Spatial Sampling',
 'Average Temporal Sampling',
 'Quality Report Filename',
 'Input Dataset Parameters Part ID',
 'Input Dataset Observe Frames Part ID',
 'Summit Software Version',
 'Calibration Workflow Name',
 'Calibration Workflow Version',
 'HDU Creation Date',
 'Observing Program Execution ID',
 'Instrument Program Execution ID',
 'Header Specifica

In [51]:
vbi['Input Dataset Observe Frames Part ID']

252
251
242
244
253
246
...
231
232
250
249
